# The Dreambooth Ion Cannon

    The first thing I need to say is that this is not original work.
    
    It is based off of "The Repo Formerly Known As Dreambooth" by Joe Penna and his many extraordinarily talented collaborators, and further tweaks to the training process have been made by my friend Kane Wallmann. I am in awe of them, and owe them all a bottle of something from the top shelf.
    
    Dreambooth is a phenomenal piece of work, despite being described as 'walking through a jungle without a torch'.
    
    This variant of the repository is not an "improvement" per se - rather, it is a simplification of the process of
    image generation by virtue of abstracting away details such as requiring you to interface with external sites, and
    providing some prompts that I've found work quite well across the board.
    
    The cost to this abstraction is a significantly larger base repository, coming out at 4.8 gigabytes. Them's the breaks.
    
    There is still a level of technical know-how required, however you - the intended audience - are unlikely to be
    reading this unless you've already followed part of the guide in the README for this repository fork, so you're
    already most of the way there.
    
    Some provisos:
        * I have bundled Stable Diffusion 1.4 in the repository containing this notebook. Please read the license 
          file `STABLE DIFFUSION LICENSE.md` in order to understand what you implicitly agree to by making use of this tech.
          The long and short of it is that so long as you don't approach this tool with ill intent, you're fine.
        * Despite Kane's adjustments enabling multi-subject training, this repo *at present* is designed to only accept 
          one subject at a time - that's what the prompts are expecting, so training multiple people is a waste. In future
          iterations of this repository, I will introduce some branching logic allowing you to select how many people you
          want to include in the generated images.
          
    I hope that that this inspires a little bit of wonder in you.
    
    /Laurence

# Naming And Classification

\*\*\* This is the **only** cell you have to edit. \*\*\* 

Before you opened this notebook per instructions from the repository, you should have uploaded 8-10 .png photographs of the person you are trying to generate images for into the directory `/training_samples/{class}` with filenames:

* `name class_001.png`
* `name class_001.png`
* ...
* `name class_00n.png`

If you haven't done this, go back to the README of this repository (scroll down from the link you were given) and give it a go.

Your choices for class are `man`, `woman` or `person`, whereas name is dealers choice, but do keep it as a single word (i.e. no spaces).

**For example**, if you're trying to train on Princess Diana, your files would be called `princessdiana woman_001.png` et cetera.

The spacing and underscores are important here, so please double-check.

----

In the cell below, edit the variables to reflect the `name` and `class` you have chosen. _Again_, please check for accuracy.

In [1]:
target_name = "chrissy"
target_class = "woman"

# Now click *Cell > Run All Below* from the menu bar above.

## Go away for 90 minutes.

# Build Environment

In [2]:
#BUILD ENV
!pip install omegaconf
!pip install einops
!pip install pytorch-lightning==1.6.5
!pip install test-tube
!pip install transformers
!pip install kornia
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install setuptools==59.5.0
!pip install pillow==9.0.1
!pip install torchmetrics==0.6.0
!pip install -e .
!pip install protobuf==3.20.1
!pip install gdown
!pip install pydrive
!pip install -qq diffusers["training"]==0.3.0 transformers ftfy
!pip install -qq "ipywidgets>=7,<8"
!pip install huggingface_hub
!pip install ipywidgets==7.7.1
!pip install py7zr==0.20.0
!pip install multivolumefile

Obtaining taming-transformers from git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
  Updating /home/bryan/venv/gpu/src/taming-transformers clone (to revision master)
  Running command git fetch -q --tags
  Running command git reset --hard -q 24268930bf1dce879235a7fddd0b2355b84d7ea6
  Preparing metadata (setup.py) ... done
  Attempting uninstall: taming-transformers
    Found existing installation: taming-transformers 0.0.1
    Uninstalling taming-transformers-0.0.1:
      Successfully uninstalled taming-transformers-0.0.1
  Running setup.py develop for taming-transformers
Obtaining clip from git+https://github.com/openai/CLIP.git@main#egg=clip
  Updating /home/bryan/venv/gpu/src/clip clone (to revision main)
  Running command git fetch -q --tags
  Running command git reset --hard -q d50d76daa670286dd6cacf3bcd80b5e4823fc8e1
  Preparing metadata (setup.py) ... done


  Attempting uninstall: clip
    Found existing installation: clip 1.0
    Uninstalling clip-1.0:
      Successfully uninstalled clip-1.0
  Running setup.py develop for clip
Obtaining file:///home/bryan/Downloads/Dreambooth-Ion-Cannon
  Preparing metadata (setup.py) ... done
  Attempting uninstall: latent-diffusion
    Found existing installation: latent-diffusion 0.0.1
    Uninstalling latent-diffusion-0.0.1:
      Successfully uninstalled latent-diffusion-0.0.1
  Running setup.py develop for latent-diffusion
  Using cached protobuf-3.20.1-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.10.1 requires protobuf<3.20,>=3.9

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 2.10.1 requires protobuf<5.0.0dev,>=3.20.1, but you have protobuf 3.19.6 which is incompatible.


In [3]:
REPO_PATH="."
# REPO_PATH="/workspace/Dreambooth-Ion-Cannon"

# Decompressing/Reconstituting Stable Diffusion v1.4

In [6]:
import multivolumefile
import py7zr
with multivolumefile.open('base_model/model.7z', mode='rb') as target_archive:
    with py7zr.SevenZipFile(target_archive, 'r') as archive:
        archive.extractall()

# Training and Pruning

In [4]:
target_token = f"{target_name} {target_class}"
project_name = f"{target_name}_model"
model_name = f"{project_name}.ckpt"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples 8 \
 --n_iter 8 \
 --seed 2148934030 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "{target_token}, portrait shinkai makoto studio ghibli studio key hideaki anno sakimichan stanley artgerm lau rossdraws james jean marc simonetti elegant highly detailed digital painting artstation pixiv" \
 --outloc "{target_name}_ghibli" \
 --outdir "outputs/{target_name}_ghibli"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples 8 \
 --n_iter 8 \
 --seed 1694397999 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "a beautiful portrait of {target_token}, with isolated flowers with strong dark comic outlines, colorful, psychedelic, intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, art by artgerm and greg rutkowski and alphonse mucha" \
 --outloc "{target_name}_flowers" \
 --outdir "outputs/{target_name}_flowers"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples 8 \
 --n_iter 8 \
 --seed 1337 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "digital painting of {target_token} cosmic background, scenery landscape, professional, award - winning, trending on artstation, hyper detailed, realistic, beautiful, emotional, shiny, golden, picture" \
 --outloc "{target_name}_cosmic" \
 --outdir "outputs/{target_name}_cosmic"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples 8 \
 --n_iter 8 \
 --seed 69 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "portrait of {target_token} cyberpunk neon - bordered cyborg, 7 0 mm focal length, by ilya kuvshinov, krenz cushart, Greg Rutkowski, trending on artstation sharp focus illustration, aesthetic, very inspirational, arthouse" \
 --outloc "{target_name}_cyborg" \
 --outdir "outputs/{target_name}_cyborg"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples 8 \
 --n_iter 8 \
 --seed 3491584590 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "bioshock {target_token} as big sister portrait, intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, art by artgerm and greg rutkowski and alphonse mucha" \
 --outloc "{target_name}_bioshock" \
 --outdir "outputs/{target_name}_bioshock"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples 8 \
 --n_iter 8 \
 --seed 1201562208 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "A soft and breathtaking detailed painting of a {target_token} with a crown on their head in the style of gustav klimt, shiny gold, elegant, highly detailed, artstation, fluo colors, concept art, matte, sharp focus, art by gustav klimt and alphonse mucha" \
 --outloc "{target_name}_crown" \
 --outdir "outputs/{target_name}_crown"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples 8 \
 --n_iter 8 \
 --seed 69 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "digital painting of {target_token} by greg rutkowski neon cyberpunk" \
 --outloc "{target_name}_cyberpunk" \
 --outdir "outputs/{target_name}_cyberpunk"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples 8 \
 --n_iter 8 \
 --seed 4085433209 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "Up close portrait of a beautiful {target_token} in black and white, photorealistic, upper body, art by diego fazio and diegokoi and artgerm, concept art, hyper sharp focus, 8k highly detailed" \
 --outloc "{target_name}_blackwhite" \
 --outdir "outputs/{target_name}_blackwhite"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples 8 \
 --n_iter 8 \
 --seed 2603348708 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "A beautiful portrait of a {target_token} with diamonds and glittering skin, a detailed painting by greg rutkowski and raymond swanland, featured on cgsociety, fantasy art, detailed painting, artstation hd, photorealistic, fantasy, intricate, elegant, rainbow bubbles, highly detailed, digital painting" \
 --outloc "{target_name}_ethereal" \
 --outdir "outputs/{target_name}_ethereal"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples 2 \
 --n_iter 2 \
 --seed 42 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "{target_token}" \
 --outloc "{target_name}_raw" \
 --outdir "outputs/{target_name}_raw"

# Image Generation

In [ ]:
model_path = f"/workspace/Dreambooth-Ion-Cannon/trained_models/{model_name}"

In [ ]:
last_checkpoint_file_pruned = directory_paths[-1] + "/checkpoints/last-pruned.ckpt"
!mkdir -p trained_models
!mv {last_checkpoint_file_pruned} trained_models/{model_name}

In [ ]:
# This version should automatically prune around 10GB from the ckpt file
last_checkpoint_file = directory_paths[-1] + "/checkpoints/last.ckpt"
!python "prune_ckpt.py" --ckpt {last_checkpoint_file}

In [ ]:
directory_paths = !ls -d logs/*

In [5]:
!echo {model_name}

chrissy_model.ckpt


In [8]:
!export PYTORCH_ENABLE_MPS_FALLBACK=1

In [ ]:
!rm -rf training_samples/{target_class}/.ipynb_checkpoints
!python "main.py" \
 --base configs/stable-diffusion/v1-finetune_unfrozen.yaml \
 -t \
 --actual_resume "model.ckpt" \
 --reg_data_root "{REPO_PATH}/outputs/txt2img-samples/regularization" \
 -n {project_name} \
 --gpus 0, \
 --data_root "{REPO_PATH}/training_samples" \
 --max_training_steps 1000 \
 --no-test

Global seed set to 23
Running on GPUs 0,
Loading model from model.ckpt
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 64, 64) = 16384 dimensions.
making attention of type 'vanilla' with 512 in_channels
Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.14.self_attn.q_proj.bias', 'visual_projection.weight', 'vision_model.encoder.layers.16.self_attn.v_proj.bias', 'vision_model.encoder.layers.15.mlp.fc2.weight', 'vision_model.encoder.layers.22.self_attn.q_proj.bias', 'vision_model.encoder.layers.9.self_attn.out_proj.weight', 'vision_model.encoder.layers.6.self_attn.q_proj.weight', 'vision_model.encoder.layers.23.self_attn.q_proj.weight', 'vision_model.encoder.layers.12.layer_norm1.bias', 'vision_model.encoder.layers.23.layer_norm2.weight', 'vision_model.encoder.layers.11

Restored from model.ckpt with 0 missing and 2 unexpected keys
Unexpected Keys: ['model_ema.decay', 'model_ema.num_updates']
/home/bryan/venv/gpu/lib/python3.10/site-packages/pytorch_lightning/loggers/test_tube.py:105: LightningDeprecationWarning: The TestTubeLogger is deprecated since v1.5 and will be removed in v1.7. We recommend switching to the `pytorch_lightning.loggers.TensorBoardLogger` as an alternative.
  rank_zero_deprecation(
Monitoring val/loss_simple_ema as checkpoint metric.
Merged modelckpt-cfg: 
{'target': 'pytorch_lightning.callbacks.ModelCheckpoint', 'params': {'dirpath': 'logs/training_samples2022-10-05T23-02-23_chrissy_model/checkpoints', 'filename': '{epoch:06}', 'verbose': True, 'save_last': True, 'monitor': 'val/loss_simple_ema', 'save_top_k': 1, 'every_n_train_steps': 500}}
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
#### Data #####
train, PersonalizedBatchBase, 1

In [17]:
!pip show torch

Name: torch
Version: 1.12.1
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /home/bryan/venv/gpu/lib/python3.10/site-packages
Requires: typing-extensions
Required-by: accelerate, clip, diffusers, kornia, latent-diffusion, pytorch-lightning, taming-transformers, test-tube, torchmetrics, torchvision
